In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import cv2, glob, random, shutil, os, plistlib, pickle
import pandas as pd
import pydicom as pdcm
from pathlib import Path
from sklearn.model_selection import train_test_split
import albumentations as A
from skimage.draw import polygon
import matplotlib.pyplot as plt
from read_roi import read_roi_file

from preprocessing_utils import *

random.seed(0)
np.random.seed(0)

In [2]:
DCM_PATH = '/home/aminul/CVL/data/INbreast_Release_1.0/AllDICOMs/'
XML_PATH = '/home/aminul/CVL/data/INbreast_Release_1.0/AllXML/'
ROI_PATH = '/home/aminul/CVL/data/INbreast_Release_1.0/AllROI/'

In [3]:
inbreast_data = pd.read_csv('Inbreast_simple.csv')

### Getting data with BIRADs 2 or more

In [6]:
filename, mass_label, cal_label, bm_label = [],[],[],[]

for i in range(len(inbreast_data)-1):
    
    if inbreast_data['Abnormal_label'][i] != 0:
    
        filename.append(int(inbreast_data['File_Name'][i]))

        bm_label.append(inbreast_data['MB_label'][i])
        
        if inbreast_data['Mass'][i] == 1:
            mass_label.append(1)
        else: 
            mass_label.append(0)
            
        if inbreast_data['Micros'][i] == 1:
            cal_label.append(1)
        else:
            cal_label.append(0)
        

### Original Image and Mask Preparation

In [ ]:
for k in range(0,343):
    img_name = str(filename[k])
    dcm = pdcm.dcmread(glob.glob(os.path.join(DCM_PATH,str(img_name)+'*.dcm'))[0])
    img = dcm.pixel_array
    a   = Annotation(XML_PATH, img_name, img.shape)
    
    mass_mask = a.mask[:,:,0]
    cal_mask = a.mask[:,:,1]
    
    comb_mask = np.zeros(mass_mask.shape)

    for i in range(mass_mask.shape[0]):
        for j in range(mass_mask.shape[1]):
            if mass_mask[i][j] == 1 or cal_mask[i][j] == 1:
                comb_mask[i][j] = 1
            else: 
                comb_mask[i][j] = 0    
                
                
    plt.imsave('./original_image/'+str(filename[k])+'.jpg', img)
    if mass_label[k] != 0:
        plt.imsave('./mass_mask/'+str(filename[k])+'_mass.jpg', mass_mask)
    if cal_label[k] != 0:
        plt.imsave('./calcification_mask/'+str(filename[k])+'_calc.jpg', cal_mask)
    plt.imsave('./combined_mask/'+str(filename[k])+'_comb.jpg', comb_mask)

In [13]:
print(len(os.listdir('original_image/')))
print(len(os.listdir('mass_mask/')))
print(len(os.listdir('calcification_mask/')))
print(len(os.listdir('combined_mask/')))

343
108
308
343


In [ ]:
patient_id = int(filename[150])
image_path = glob.glob(os.path.join(DCM_PATH,str(patient_id)+'*.dcm'))
print(image_path, patient_id)
mass_mask = load_inbreast_mask(os.path.join(XML_PATH,str(patient_id)+'.xml'))
print(mass_mask.shape)

In [ ]:
read_roi_file(os.path.join(ROI_PATH,str(patient_id)+'.roi'))

In [ ]:
ds = dicom.dcmread(image_path[0])
pixel_array_numpy = ds.pixel_array

In [ ]:
breast, mask, ground_truth_cropped = crop(pixel_array_numpy, mass_mask)
print(breast.shape)

plt.figure(figsize = (15,8))
plt.subplot(1,3,1)
plt.imshow(breast,cmap='gray')
plt.subplot(1,3,2)
plt.imshow(mask,cmap='gray')
plt.subplot(1,3,3)
plt.imshow(ground_truth_cropped,cmap='gray')

In [ ]:
normalized = truncation_normalization(breast, mask)
print(normalized.shape)

plt.figure(figsize = (15,8))
plt.imshow(normalized,cmap='gray')

In [ ]:
cl1 = clahe(normalized, 1.0)
cl2 = clahe(normalized, 2.0)

print(cl1.shape, cl2.shape)

plt.figure(figsize = (15,8))

plt.subplot(1,3,1)
plt.imshow(breast,cmap='gray')
plt.subplot(1,3,2)
plt.imshow(cl1,cmap='gray')
plt.subplot(1,3,3)
plt.imshow(cl2,cmap='gray')

In [ ]:
synthetized = cv2.merge((np.array(normalized*255, dtype=np.uint8),cl1,cl2))
print(synthetized.shape)

plt.figure(figsize = (15,8))
plt.subplot(1,3,1)
plt.imshow(breast,cmap='gray')
plt.subplot(1,3,2)
plt.imshow(synthetized,cmap='gray')
plt.subplot(1,3,3)
plt.imshow(ground_truth_cropped,cmap='gray')

In [ ]:
ground_truth_cropped.sum(), mass_mask.sum()